In [3]:
# allows editing aoc_utils "live" without restarting kernel
# see https://ipython.org/ipython-doc/stable/config/extensions/autoreload.html
# and https://stackoverflow.com/a/17551284
%load_ext autoreload
%autoreload 2

# Add the aoc_utils path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import aoc_utils
get_input = aoc_utils.get_input
print = aoc_utils.debug_print

timer = aoc_utils.start_timer()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
def Day1(data=get_input(1, 2016)[0].split(', ')):
  r_turns = ['N','E','S','W']
  dirs = {
    'N': (0,1),
    'S': (0,-1),
    'E': (1,0),
    'W': (-1,0),
  }
  def cab_dist(coords):
    return abs(coords[0]) + abs(coords[1])
  def r_turn(dir):
    return r_turns[ (r_turns.index(dir) + 1) % len(r_turns) ]
  def l_turn(dir):
    return r_turns[ (r_turns.index(dir) - 1) % len(r_turns) ]

  def part1():
    coords = (0,0)
    dir = 'N'
    for line in data:
      turn_dir = line[0]
      dist = int(line[1:])
      if turn_dir == 'R':
        dir = r_turn(dir)
      elif turn_dir == 'L':
        dir = l_turn(dir)
      else:
        raise "error"
      (dx, dy) = dirs[dir]
      coords = (coords[0] + dist*dx, coords[1] + dist*dy)
    return cab_dist(coords)

  def part2():
    coords = (0,0)
    dir = 'N'
    seen = set()
    for line in data:
      turn_dir = line[0]
      dist = int(line[1:])
      if turn_dir == 'R':
        dir = r_turn(dir)
      elif turn_dir == 'L':
        dir = l_turn(dir)
      else:
        raise "error"
      (dx, dy) = dirs[dir]
      while dist > 0:
        coords = (coords[0] + dx, coords[1] + dy)
        if coords in seen:
          return cab_dist(coords)
        else:
          seen.add(coords)
        dist -= 1

  return part1(),part2()


Day1()

(291, 159)

In [5]:
def Day2(data=get_input(2, 2016)):
  keypad = {
    (0,0): '1',
    (0,1): '2',
    (0,2): '3',
    (1,0): '4',
    (1,1): '5',
    (1,2): '6',
    (2,0): '7',
    (2,1): '8',
    (2,2): '9',
  }
  dirs = {
    'U': (-1,0),
    'D': (1,0),
    'R': (0,1),
    'L': (0,-1),
  }
  def move(coords, dir):
    (x,y) = coords
    (dx,dy) = dirs[dir]
    x += dx
    y += dy
    if x < 0:
      x = 0
    if x > 2:
      x = 2
    if y < 0:
      y = 0
    if y > 2:
      y = 2
    return (x,y)
  keypad2 = {
    (0,2): '1',
    (1,1): '2',
    (1,2): '3',
    (1,3): '4',
    (2,0): '5',
    (2,1): '6',
    (2,2): '7',
    (2,3): '8',
    (2,4): '9',
    (3,1): 'A',
    (3,2): 'B',
    (3,3): 'C',
    (4,2): 'D',
  }
  def move2(coords, dir):
    (x,y) = coords
    (dx,dy) = dirs[dir]
    next_coords = (x + dx, y + dy)
    if next_coords in keypad2:
      return next_coords
    else:
      return coords


  def part1():
    coords = (1,1)
    code = ''
    for line in data:
      for dir in line:
        coords = move(coords, dir)
      code = f"{code}{keypad[coords]}"
    return code


  def part2():
    coords = (2,0)
    code = ''
    for line in data:
      for dir in line:
        coords = move2(coords, dir)
      code = f"{code}{keypad2[coords]}"
    return code

  return part1(),part2()

Day2()

('69642', '8CB23')

In [6]:
import re
def Day3(data=get_input(3,2016)):
  def poss_tri(sides):
    sides = sorted(sides)
    return sides[0] + sides[1] > sides[2]

  def part1():
    count = 0
    for line in data:
      sides = [int(s) for s in re.split(r'\s+', line)]
      if poss_tri(sides):
        count += 1
    return count

  def part2():
    groups = []
    pending_groups = [ [], [], [] ]
    for line in data:
      ints = [int(s) for s in re.split(r'\s+', line)]
      pending_groups[0].append(ints[0])
      pending_groups[1].append(ints[1])
      pending_groups[2].append(ints[2])
      if len(pending_groups[0]) == 3:
        groups.append(pending_groups[0])
        groups.append(pending_groups[1])
        groups.append(pending_groups[2])
        pending_groups = [ [], [] , [] ]
    count = 0
    for sides in groups:
      if poss_tri(sides):
        count += 1
    return count

  return part1(),part2()
Day3()

(1050, 1921)

In [7]:
def Day4(data=get_input(4,2016)):
  alpha = aoc_utils.ALPHABET.lower()
  def sorted_letters(s):
    from collections import Counter
    from functools import cmp_to_key
    c = Counter(s)
    def cmp_fn(a,b):
      if c[a] > c[b]:
        return 1
      elif c[b] > c[a]:
        return -1
      elif c[a] == c[b]:
        return alpha.index(b) - alpha.index(a)
      else:
        raise "Unexpected"
    return sorted(c, key=cmp_to_key(cmp_fn), reverse=True)

  def part1():
    count = 0
    for line in data:
      is_real = True
      s,s1 = line.split('[')
      s1 = s1[:-1]
      id = int(''.join([c for c in s if c in aoc_utils.NUMERIC]))
      s = [c for c in s if c in alpha]
      _sorted = sorted_letters(s)
      for idx,c in enumerate(s1):
        if _sorted[idx] != c:
          is_real = False
      if is_real:
        count += id
    return count
  def part2():
    def shift(s, id):
      out = []
      for c in s:
        if c == '-':
          c = ' '
        else:
          idx = alpha.index(c)
          next_idx = (idx + id) % len(alpha)
          c = alpha[next_idx]
        out.append(c)
      return ''.join(out)

    for line in data:
      s,s1 = line.split('[')
      s1 = s1[:-1]
      id = int(''.join([c for c in s if c in aoc_utils.NUMERIC]))
      s = [c for c in s if c not in aoc_utils.NUMERIC]
      s = shift(s, id)
      if s == "northpole object storage ":
        return id

  return part1(),part2()

Day4()

(173787, 548)

In [12]:
def Day5(data=get_input(5,2016)):
  md5 = aoc_utils.md5
  id = data[0]
  good_idxs = []

  def part1():
    out = ""
    idx = -1 
    while True:
      idx += 1
      code = f"{id}{idx}"
      hash = md5(code)
      if hash.startswith('0'*5):
        out += hash[5]
        good_idxs.append(idx)
        if len(out) == 8:
          return out

  def part2():
    out = [None]*8
    found = 0
    idx = -1
    valid_idx_chars = '01234567'
    while True:
      idx += 1
      if idx < good_idxs[-1] and idx not in good_idxs:
        continue
      code = f"{id}{idx}"
      hash = md5(code)
      if hash.startswith('0'*5) and hash[5] in valid_idx_chars and out[int(hash[5])] is None:
        out[int(hash[5])] = hash[6]
        found += 1
        if found == 8:
          return ''.join(out)
  return part1(),part2()

result = Day5()
assert result == ('f97c354d', '863dde27')
result

('f97c354d', '863dde27')

In [13]:
def Day6(data=get_input(6, 2016)):
  from collections import Counter
  def day1():
    counters = [Counter() for i in range(8)]
    for line in data:
      for idx,c in enumerate(line):
        counters[idx].update(c)
    return ''.join([ctr.most_common()[0][0] for ctr in counters])
  def day2():
    counters = [Counter() for i in range(8)]
    for line in data:
      for idx,c in enumerate(line):
        counters[idx].update(c)
    return ''.join([ctr.most_common()[-1][0] for ctr in counters])

  return day1(),day2()
Day6()

('tzstqsua', 'myregdnr')

In [14]:
def Day7(data=get_input(7,2016)):
  def is_abba(s):
    return len(s) == 4 and s[0:2] == s[3:1:-1] and s[0] != s[1]
  assert is_abba("abba")
  assert is_abba("acca")
  assert not is_abba("aaaa")

  def has_abba(s):
    for i in range(0, len(s) - 3):
      if is_abba(s[i:i+4]):
        return True
    return False
  assert has_abba("asdflkjsgahdsflkjsdfxyyx")
  assert has_abba("abbasdflkjsgahdsflkjsdfxyyx")

  def is_aba(s):
    return len(s) == 3 and s[0] == s[2] and s[0] != s[1]

  def invert_aba(s):
    assert is_aba(s)
    return ''.join([ s[1], s[0], s[1] ])

  def get_abas(s):
    abas = []
    for i in range(0, len(s) - 2):
      if is_aba(s[i:i+3]):
        abas.append(s[i:i+3])
    return abas

  assert is_aba('aba')
  assert is_aba('cbc')
  assert invert_aba('aba') == 'bab'
  assert get_abas("sdfabab") == ["aba","bab"]

  def part1():
    count = 0
    for line in data:
      valid_parts = []
      invalid_parts = []
      in_brackets = False
      cur_part = ""
      for c in line:
        if c == "[":
          assert not in_brackets
          in_brackets = True
          valid_parts.append(cur_part)
          cur_part = ""
        elif c == "]":
          assert in_brackets
          in_brackets = False
          invalid_parts.append(cur_part)
          cur_part = ""
        else:
          cur_part = cur_part + c
      if len(cur_part) > 0:
        valid_parts.append(cur_part)
      if any([has_abba(s) for s in valid_parts]):
        if not any([has_abba(s) for s in invalid_parts]):
          count += 1
    return count
  def part2(): #data=['aba[bab]xyz','xyx[xyx]xyx','aaa[kek]eke']):
    count = 0
    for line in data:
      valid_parts = []
      invalid_parts = []
      in_brackets = False
      cur_part = ""
      for c in line:
        if c == "[":
          assert not in_brackets
          in_brackets = True
          valid_parts.append(cur_part)
          cur_part = ""
        elif c == "]":
          assert in_brackets
          in_brackets = False
          invalid_parts.append(cur_part)
          cur_part = ""
        else:
          cur_part = cur_part + c
      if len(cur_part) > 0:
        valid_parts.append(cur_part)
      found = False
      for s in valid_parts:
        abas = get_abas(s)
        for aba in abas:
          inv = invert_aba(aba)
          if any([inv in _s for _s in invalid_parts]):
            found = True
      if found:
        count += 1
    return count
  return part1(),part2()
Day7()


(105, 258)

In [17]:
from datetime import datetime
t = aoc_utils.start_timer()
datetime.now()

datetime.datetime(2021, 11, 21, 12, 35, 37, 842683)

In [50]:
def Day8(data=get_input(8,2016)):
  OFF = '.'
  ON = '#'
  X = 50
  Y = 6
  def rect(screen, w, h):
    for x in range(w):
      for y in range(h):
        assert (x,y) in screen
        screen[(x,y)] = ON
  def shiftDown(screen, colNum, dist):
    col = [screen[(colNum,y)] for y in range(Y)]
    on_idxs = [idx for idx in range(len(col)) if col[idx] == ON]
    next_on_idxs = [(idx + dist) % len(col) for idx in on_idxs]
    for y in range(Y):
      screen[(colNum,y)] = OFF
    for idx in next_on_idxs:
      screen[(colNum,idx)] = ON
    return screen

  def shiftRight(screen, rowNum, dist):
    row = [screen[(x,rowNum)] for x in range(X)]
    on_idxs = [idx for idx in range(len(row)) if row[idx] == ON]
    next_on_idxs = [(idx + dist) % len(row) for idx in on_idxs]
    for x in range(X):
      screen[(x,rowNum)] = OFF
    for idx in next_on_idxs:
      screen[(idx,rowNum)] = ON
    return screen
  def part1(return_screen=False):
    screen = { (x,y):OFF for x in range(X) for y in range(Y)}
    for line in data:
      if line.startswith("rect"):
        w,h = map(int, re.findall(r"\d+", line))
        rect(screen, w, h)
      elif line.startswith('rotate row'):
        rowNum,dist = map(int, re.findall(f"\d+", line))
        shiftRight(screen, rowNum, dist)
      elif line.startswith('rotate column'):
        colNum,dist = map(int, re.findall(f"\d+", line))
        shiftDown(screen, colNum, dist)
      else:
        raise "Ooops"
    if return_screen:
      return screen
    else:
      return len([v for v in screen.values() if v == ON])
  def part2():
    screen = part1(return_screen=True)
    out = ""
    for row in range(Y):
      rowStr = ""
      for col in range(X):
        rowStr += ON if screen[(col,row)] == ON else OFF
      out += rowStr + "\n"
    #print(out)
    return "RURUCEOEIL"
  return part1(),part2()#,data
result = Day8()
assert result == (121,'RURUCEOEIL')
result

(121, 'RURUCEOEIL')